# Lab | Text Generation from Shakespeare's Sonnet

This notebook explores the fascinating domain of text generation using a deep learning model trained on Shakespeare's sonnets. 

The objective is to create a neural network capable of generating text sequences that mimic the style and language of Shakespeare.

By utilizing a Recurrent Neural Network (RNN) with Long Short-Term Memory (LSTM) layers, this project aims to demonstrate how a model can learn and replicate the complex patterns of early modern English. 

The dataset used consists of Shakespeare's sonnets, which are preprocessed and tokenized to serve as input for the model.

Throughout this notebook, you will see the steps taken to prepare the data, build and train the model, and evaluate its performance in generating text. 

This lab provides a hands-on approach to understanding the intricacies of natural language processing (NLP) and the potential of machine learning in creative text generation.

Let's import necessary libraries

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np

Let's get the data!

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/martin-gorner/tensorflow-rnn-shakespeare/master/shakespeare/sonnets.txt'
resp = requests.get(url)
with open('sonnets.txt', 'wb') as f:
    f.write(resp.content)

data = open('sonnets.txt').read()

corpus = data.lower().split("\n")

Step 1: Initialise a tokenizer and fit it on the corpus variable using .fit_on_texts

In [ ]:
# Your code here :

# Initialize the tokenizer
tokenizer = Tokenizer()

#Fit the tokenizer on the corpus
tokenizer.fit_on_texts(corpus)

word_index = tokenizer.word_index
print("Word Index:\n", word_index)

Step 2: Calculate the Vocabulary Size

Let's figure out how many unique words are in your corpus. This will be the size of your vocabulary.

Calculate the length of tokenizer.word_index, add 1 to it and store it in a variable called total_words.

In [ ]:
# Your code here :

total_words = len(tokenizer.word_index) + 1
print("Total Vocabulary Size:", total_words)


Create an empty list called input_sequences.

For each sentence in your corpus, convert the text into a sequence of integers using the tokenizer.
Then, generate n-gram sequences from these tokens.

Store the result in the list input_sequences.

In [ ]:
# Your code here :

# Step 3: Create input_sequences list
input_sequences = []

# Loop through each sentence in the corpus
for sentence in corpus:
    
    # Convert the sentence into a sequence of integers
    token_list = tokenizer.texts_to_sequences([sentence])[0]
    
    # Generate n-gram sequences
    for i in range(1, len(token_list) + 1):
        n_gram_sequence = token_list[:i] 
        input_sequences.append(n_gram_sequence)


print("Input Sequences:")
for seq in input_sequences:
    print(seq)


Calculate the length of the longest sequence in input_sequences. Assign the result to a variable called max_sequence_len.

Now pad the sequences using pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre').
Convert it to a numpy array and assign the result back to our variable called input_sequences.

In [ ]:
# Your code here :

from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

#Calculate the maximum sequence length
max_sequence_len = max(len(seq) for seq in input_sequences)
print("Maximum Sequence Length:", max_sequence_len)


In [ ]:

#Pad the sequences
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Convert to a numpy array
input_sequences = np.array(input_sequences)

# Print the padded sequences
print("\nPadded Input Sequences:")
print(input_sequences)


Prepare Predictors and Labels

Split the sequences into two parts:

- Predictors: All elements from input_sequences except the last one.
- Labels: The last element of each sequence in input_sequences.

In [ ]:
# Your code here :

#Prepare predictors and labels
predictors = input_sequences[:, :-1]
labels = input_sequences[:, -1]

# Print predictors and labels
print("Predictors (X):")
print(predictors)

print("\nLabels (y):")
print(labels)


One-Hot Encode the Labels :

Convert the labels (which are integers) into one-hot encoded vectors. 

Ensure the length of these vectors matches the total number of unique words in your vocabulary.

Use ku.to_categorical() on labels with num_classes = total_words

Assign the result back to our variable labels.

In [ ]:
# Your code here :

from tensorflow.keras.utils import to_categorical

#One-hot encode the labels
labels = to_categorical(labels, num_classes=total_words)

# Print the one-hot encoded labels
print("One-Hot Encoded Labels:")
print(labels)


# Initialize the Model

Start by creating a Sequential model.

Add Layers to the Model:

Embedding Layer: The first layer is an embedding layer. It converts word indices into dense vectors of fixed size (100 in this case). Set the input length to the maximum sequence length minus one, which corresponds to the number of previous words the model will consider when predicting the next word.

Bidirectional LSTM Layer: Add a Bidirectional LSTM layer with 150 units. This layer allows the model to learn context from both directions (past and future) in the sequence. return_sequences=True

Dropout Layer: Add a dropout layer with a rate of 0.2 to prevent overfitting by randomly setting 20% of the input units to 0 during training.

LSTM Layer: Add a second LSTM layer with 100 units. This layer processes the sequence and passes its output to the next layer.

Dense Layer (Intermediate): Add a dense layer with half the total number of words as units, using ReLU activation. A regularization term (L2) is added to prevent overfitting.

Dense Layer (Output): The final dense layer has as many units as there are words in the vocabulary, with a softmax activation function to output a probability distribution over all words.

In [ ]:

model = Sequential([])

# Your code here :
    
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.regularizers import l2


# Embedding Layer
model.add(Embedding(
    input_dim=total_words,
    output_dim=100,
    input_length=max_sequence_len - 1
))

# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(150, return_sequences=True)))

# Dropout Layer
model.add(Dropout(0.2))

# Second LSTM Layer
model.add(LSTM(100))

# Dense Layer (Intermediate)
model.add(Dense(total_words // 2, activation='relu', kernel_regularizer=l2(0.01)))

# Dense Layer (Output)
model.add(Dense(total_words, activation='softmax'))

# Print model summary
model.summary()


# Compile the Model:

Compile the model using categorical crossentropy as the loss function, the Adam optimizer for efficient training, and accuracy as the metric to evaluate during training.

In [ ]:
# Your code here :

from tensorflow.keras.optimizers import Adam

#Compile the model
model.compile(
    loss='categorical_crossentropy', 
    optimizer=Adam(learning_rate=0.001), 
    metrics=['accuracy']
)

print("Model compiled successfully!")


# Print Model Summary:

Use model.summary() to print a summary of the model, which shows the layers, their output shapes, and the number of parameters.

In [ ]:
# Your code here :

#Print model summary
model.summary()


# Now train the model for 50 epochs and assign it to a variable called history.

Training the model with 50 epochs should get you around 40% accuracy.

You can train the model for as many epochs as you like depending on the time and computing constraints you are facing. Ideally train it for a larger amount of epochs than 50.

That way you will get better text generation at the end.

However, dont waste your time.

In [ ]:
# Your code here :

# Step 11: Train the model
history = model.fit(
    predictors,
    labels,
    epochs=50,
    verbose=1
)

# Print a message to confirm completion
print("Model training complete!")


# Use plt from matplotlib to plot the training accuracy over epochs and the loss over epochs

First you will have to get the accuracy and loss data over epochs, you can do this by using methods on your model.

In [ ]:
# Your code here :
import matplotlib.pyplot as plt

# Access accuracy and loss from the history object
accuracy = history.history['accuracy']
loss = history.history['loss']




plt.figure(figsize=(12, 6))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(accuracy, label='Accuracy', color='blue')
plt.title('Training Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(loss, label='Loss', color='red')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Generate text with the model based on a seed text

Now you will create two variables :

- seed_text = 'Write the text you want the model to use as a starting point to generate the next words'
- next_words = number_of_words_you_want_the_model_to_generate

Please change number_of_words_you_want_the_model_to_generate by an actual integer.

In [ ]:
# Your code here :

seed_text = "Shall I compare thee to a summer's day"
next_words = 10


Now create a loop that runs based on the next_words variable and generates new text based on your seed_text input string. Print the full text with the generated text at the end.

This time you dont get detailed instructions.

Have fun!

In [ ]:
# Your code here :

# Define seed text and number of words to generate
seed_text = "Shall I compare thee to a summer's day"
next_words = 10

# Loop to generate words
for _ in range(next_words):

 # Convert seed_text to a sequence of integers
    token_list = tokenizer.texts_to_sequences([seed_text])[0]

 # Pad the sequence to match model input size
    token_list = np.array(pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre'))

# Predict the next word
    predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
    
    output_word = [word for word, index in tokenizer.word_index.items() if index == predicted]
    output_word = output_word[0] if output_word else ''
    
    seed_text += ' ' + output_word

# Print the final generated text
print("Generated Text:")
print(seed_text)



Experiment with at least 3 different seed_text strings and see what happens!

In [ ]:
# Your code here :

# Define different seed texts
seed_texts = [
    "Shall I compare thee to a summer's day",
    "The rough winds do shake the darling buds",
    "And summer's lease hath all too short"
]

next_words = 10

# Experiment with different seed texts
for i, seed_text in enumerate(seed_texts):
    print(f"\nExperiment {i+1}: Seed Text -> '{seed_text}'")
    
    generated_text = seed_text
    for _ in range(next_words):
        
        # Tokenize and pad the current seed_text
        token_list = tokenizer.texts_to_sequences([generated_text])[0]
        token_list = np.array(pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre'))
        
        # Predict the next word
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        
        # Find the word corresponding to the predicted index
        output_word = [word for word, index in tokenizer.word_index.items() if index == predicted]
        output_word = output_word[0] if output_word else ''
        
        # Append the predicted word to the generated text
        generated_text += ' ' + output_word
    
    # Print the full generated text
    print("Generated Text:")
    print(generated_text)
    print("-" * 80)
